In [ ]:
import pandas as pd

df_preview = pd.read_csv(news_csv, nrows=5)
# Display the first row (article) of the dataframe
# print(df_preview.iloc[0, 7]) # text is in the 8th column use .iloc[row, column]

# print the last row (article) of the dataframe
# print(df_preview.iloc[-1, 7]) # text is in the 8th column use .iloc[row, column]


In [ ]:
import os
import requests
import zipfile
import pandas as pd
import csv

DATA_DIR = "../data/raw/FNSPID"
os.makedirs(DATA_DIR, exist_ok=True)

# ----------------------------
# Step 1: Download Stock History Zip (already done)
# ----------------------------
# stock_url = "https://huggingface.co/datasets/Zihan1004/FNSPID/resolve/main/Stock_price/full_history.zip"
# stock_zip = os.path.join(DATA_DIR, "full_history.zip")
# download_file(stock_url, stock_zip)

needed_tickers = ["AAPL.csv", "MSFT.csv", "GOOG.csv", "AMZN.csv"]
# with zipfile.ZipFile(stock_zip, "r") as z:
#     for file in needed_tickers:
#         if file in z.namelist():
#             z.extract(file, DATA_DIR)

# ----------------------------
# Step 2: Download News File (already done — you have 5.97GB CSV)
# ----------------------------
# news_url = "https://huggingface.co/datasets/Zihan1004/FNSPID/resolve/main/Stock_news/nasdaq_exteral_data.csv"
news_csv = os.path.join(DATA_DIR, "nasdaq_exteral_data.csv")


# ----------------------------
# Step 3: Filter News by Tickers + Date (STREAMING)
# ----------------------------
output_file = os.path.join(DATA_DIR, "news_filtered.csv")

tickers = {"AAPL", "MSFT"}
start_date = pd.to_datetime("2020-09-16")
end_date = pd.to_datetime("2025-09-15")

with open(news_csv, "r", encoding="utf-8") as f_in, \
     open(output_file, "w", encoding="utf-8", newline="") as f_out:

    reader = csv.DictReader(f_in)
    writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames)
    writer.writeheader()

    kept = 0
    processed = 0
    for row in reader:
        processed += 1
        if row["Stock_symbol"] not in tickers:
            continue
        try:
            d = pd.to_datetime(row["Date"]).tz_localize(None)
        except Exception:
            continue
        if start_date <= d <= end_date:
            writer.writerow(row)
            kept += 1

        # log every 1M rows so you see progress
        if processed % 1_000_000 == 0:
            print(f"Processed {processed:,} rows… kept {kept:,}")

print(f"✅ Done! Filtered rows saved to {output_file}")
print(f"📊 Total rows kept: {kept}")


KeyboardInterrupt: 

In [ ]:
# # see if Google symbol is present in the original file
# import csv

# news_csv = "../data/raw/FNSPID/nasdaq_exteral_data.csv"

# found = set()
# with open(news_csv, "r", encoding="utf-8") as f:
#     reader = csv.DictReader(f)
#     for i, row in enumerate(reader):
#         sym = row["Stock_symbol"]
#         if "MSFT" in sym or "MSFT" in sym:
#             found.add(sym)
#         if i > 2_000_000:  # stop after 2M rows for speed
#             break

# print("✅ Found MSFT symbols in file:", found)


✅ Found MSFT symbols in file: set()


In [ ]:
# # see how GOOGL, and MSFT are represented in the original file
# import csv

# symbols = set()
# with open("../data/raw/FNSPID/nasdaq_exteral_data.csv", "r", encoding="utf-8") as f:
#     reader = csv.DictReader(f)
#     for i, row in enumerate(reader):
#         sym = row["Stock_symbol"]
#         if "MSFT" in sym or "GOOG" in sym:
#             symbols.add(sym)
#         if i > 2_000_000:  # stop early for speed
#             break

# print("Symbols found for MSFT/GOOG:", symbols)


Symbols found for MSFT/GOOG: set()


In [50]:
df = pd.read_csv("C:/Users/User/Desktop/Data Science/Fall 2025/Generative AI/Project/genai-financial-doc-analysis/data/raw/FNSPID/news_filtered.csv")

print(df.iloc[1, 4]) # text is in the 8th column use .iloc[row, column]
print(len(df))

https://www.nasdaq.com/articles/brokers-suggest-investing-in-apple-aapl%3A-read-this-before-placing-a-bet
13647


In [ ]:
### Option 2:
# Select keywords, ticker names, sectors, and download only those rows fro the dataset which should be a lot faster. Download all values of a row if the criteria is met.

# Regex friendly eyword list for filtering news

# Ticker/sector alignment so each article gets linked to a stock ticker

In [ ]:
# # ----------------------------
# # 01 - Data Collection (News Section)
# # ----------------------------

# import os
# import pandas as pd
# import re
# import requests

# # ----------------------------
# # Setup paths
# # ----------------------------
# DATA_DIR = "../data/raw/FNSPID"
# os.makedirs(DATA_DIR, exist_ok=True)

# news_url = "https://huggingface.co/datasets/Zihan1004/FNSPID/resolve/main/Stock_news/nasdaq_exteral_data.csv"
# news_csv = os.path.join(DATA_DIR, "nasdaq_exteral_data_selective.csv")
# filtered_news_path = os.path.join(DATA_DIR, "news_filtered.csv")

# # ----------------------------
# # Download News File (if not present)
# # ----------------------------
# # def download_file(url, out_path):
# #     if os.path.exists(out_path):
# #         print(f"✅ Already downloaded: {out_path}")
# #         return
# #     print(f"⬇️ Downloading {url} ... (this is a large file, ~8GB)")
# #     with requests.get(url, stream=True) as r:
# #         r.raise_for_status()
# #         with open(out_path, "wb") as f:
# #             for chunk in r.iter_content(chunk_size=8192):
# #                 if chunk:
# #                     f.write(chunk)
# #     print(f"✅ Saved to {out_path}")

# # download_file(news_url, news_csv)

# # ----------------------------
# # Define Filtering Keywords
# # ----------------------------
# tickers = ["AAPL","MSFT","GOOG","AMZN"]

# keywords = [
#     r"\binflation\b", r"\bdeflation\b", 
#     r"\binterest rate(s)?\b", r"\brate (hike|cut)\b", 
#     r"\bfederal reserve\b", r"\bfed\b", 
#     r"\bquantitative easing\b", r"\btightening\b", 
#     r"\brecession\b", r"\bgdp (growth|contraction)\b", 
#     r"\bunemployment\b", r"\bjobless claims?\b", 
#     r"\belection\b", r"\bgovernment shutdown\b", 
#     r"\btrade war\b", r"\btariffs?\b", 
#     r"\bsanctions?\b", r"\bgeopolitical tension(s)?\b", 
#     r"\boil (embargo|shock)\b", 
#     r"\bvolatility spike\b", r"\bvix\b", 
#     r"\bliquidity crisis\b", r"\bcredit downgrade\b", 
#     r"\bbank failure\b", r"\bbailout\b", 
#     r"\bdefault(s)?\b", r"\bdebt ceiling\b", 
#     r"\bearnings (miss|beat)\b", r"\bguidance cut\b", 
#     r"\bipo (surge|slump)\b", r"\b(merger|acquisition)\b", 
#     r"\bantitrust investigation\b", 
#     r"\bpandemic\b", r"\bcovid\b", 
#     r"\bsupply chain disruption\b"
# ]

# pattern = re.compile("|".join(keywords), flags=re.IGNORECASE)

# # ----------------------------
# # Filter News in Chunks (Updated for FNSPID schema)
# # ----------------------------
# print("⏳ Filtering news file for tickers and keywords...")

# chunksize = 100_000
# first = True
# rows_written = 0

# for chunk in pd.read_csv(news_csv, chunksize=chunksize, low_memory=False):
#     mask = (
#         (chunk["Stock_symbol"].isin(tickers)) |
#         (chunk["Article"].str.contains(pattern, na=False)) |
#         (chunk["Article_title"].str.contains(pattern, na=False))
#     )
#     sub = chunk[mask]

#     if not sub.empty:
#         sub.to_csv(filtered_news_path, mode="a", index=False, header=first)
#         rows_written += len(sub)
#         first = False
#         print(f"✅ Added {len(sub)} rows (total: {rows_written})")

# print("🎉 Finished filtering news!")
# print("✅ Saved filtered news to:", filtered_news_path)

# # ----------------------------
# # Quick Sanity Check
# # ----------------------------
# news_filtered = pd.read_csv(filtered_news_path, nrows=5)
# print(news_filtered[["Date","Stock_symbol","Article_title"]])



⏳ Filtering news file for tickers and keywords...


C:\Users\User\AppData\Local\Temp\ipykernel_2580\2810447847.py:79: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (chunk["Article"].str.contains(pattern, na=False)) |
C:\Users\User\AppData\Local\Temp\ipykernel_2580\2810447847.py:80: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (chunk["Article_title"].str.contains(pattern, na=False))


✅ Added 42471 rows (total: 42471)


C:\Users\User\AppData\Local\Temp\ipykernel_2580\2810447847.py:79: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (chunk["Article"].str.contains(pattern, na=False)) |
C:\Users\User\AppData\Local\Temp\ipykernel_2580\2810447847.py:80: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (chunk["Article_title"].str.contains(pattern, na=False))


✅ Added 34545 rows (total: 77016)


C:\Users\User\AppData\Local\Temp\ipykernel_2580\2810447847.py:79: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (chunk["Article"].str.contains(pattern, na=False)) |
C:\Users\User\AppData\Local\Temp\ipykernel_2580\2810447847.py:80: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (chunk["Article_title"].str.contains(pattern, na=False))


✅ Added 38147 rows (total: 115163)


C:\Users\User\AppData\Local\Temp\ipykernel_2580\2810447847.py:79: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (chunk["Article"].str.contains(pattern, na=False)) |


In [ ]:
# news_csv = r"C:\Users\User\Desktop\Data Science\Fall 2025\Generative AI\Project\genai-financial-doc-analysis\data\raw\FNSPID\nasdaq_exteral_data_selective.csv"

# df_sample = pd.read_csv(news_csv, nrows=5)
# print(df_sample.columns)
# print(df_sample.head())


Index(['Unnamed: 0', 'Date', 'Article_title', 'Stock_symbol', 'Url',
       'Publisher', 'Author', 'Article', 'Lsa_summary', 'Luhn_summary',
       'Textrank_summary', 'Lexrank_summary'],
      dtype='object')
   Unnamed: 0                     Date  \
0         0.0  2023-12-16 23:00:00 UTC   
1         1.0  2023-12-12 00:00:00 UTC   
2         2.0  2023-12-12 00:00:00 UTC   
3         3.0  2023-12-07 00:00:00 UTC   
4         4.0  2023-12-07 00:00:00 UTC   

                                       Article_title Stock_symbol  \
0  Interesting A Put And Call Options For August ...            A   
1  Wolfe Research Initiates Coverage of Agilent T...            A   
2  Agilent Technologies Reaches Analyst Target Price            A   
3  Agilent (A) Enhances BioTek Cytation C10 With ...            A   
4  Pre-Market Most Active for Dec 7, 2023 : SQQQ,...            A   

                                                 Url  Publisher  Author  \
0  https://www.nasdaq.com/articles/interesting-